In [ ]:
import os
api_key = os.environ["MISTRAL_API_KEY"]


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain


In [3]:
# Load data
loader = TextLoader("/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/well_remark.txt")
docs = loader.load()
# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)


/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [4]:
# Create the vector store 
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()

In [5]:
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [6]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "List date of running tools that encountered an obstruction, and also please include if there is any sand encountered during running"})
print(response["answer"])

Based on the provided context, the dates when tools encountered an obstruction are:

1. 20-Jul-2000: The 3" shifting tool was run and stopped due to a high angle. No mention of sand was made during this operation.
2. 10-Mar-2000: The 1-11/16" GR-CCL-Tungsten weight was run and stopped at 7150'. During this operation, sea water was pumped down, but no mention of sand was made.
3. 15-Jan-2013: A slickline tool was run and encountered an obstruction at the sliding side door (SSD). No mention of sand was made during this operation.

There is no mention of sand being encountered during any of the listed operations.
